In [ ]:
#!/usr/bin/env python3
"""
MAIN PIPELINE: Oil Palm Fruit Detection and Analysis (4-stage unified system)
- SECTION 1: Under-canopy fruit detection (YOLO)
- SECTION 2: Over-canopy zoomed frame extraction (crown-based)
- SECTION 3: Heading recalibration to bin images (based on Ground Truth to drone starting position)
- SECTION 4: Red-blob masking + analysis (grey/blob overlays)
- SECTION 5: Final scoring of Ground Truth vs Orbit results (binary + quantitative accuracy)
    Compute Orbit Score (dumb method)
    For each point's orbits:
    Mark all frames where max_blob_grey > THRESHOLD as “spikes.”
    Map each image's index to its compass section, count unique sections.
    Score 2 if ≥5 unique sections or any opposing-section pair; Score 1 if ≥1; Score 0 otherwise.
"""

# ─── IMPORTS ───────────────────────────────────────────────────────
import os
import re
import cv2
import math
import numpy as np
import contextlib
import pandas as pd
from tqdm import tqdm
from pathlib import Path
from ultralytics import YOLO

# ─── TOGGLE WHAT TO RUN ──────────────────────────────────────────────
PROCESS_IMAGES = False
PROCESS_VIDEOS = False
RECALIBRATE_HEADING = False
EXTRACT_REDBLOB = True
FINAL_ANALYSIS = True

# ─── CONFIGURATION ───────────────────────────────────────────────────
BASE_DATE = "16_7_2025"
GT_DATE = "16_7_2025"
images_per_quadrant = 7 
TARGET_TIME = 2.0    # Time in seconds to begin extracting frames from video
REFERENCE_ANGLE = 0  # North
ZOOM_FACTOR = 6      # Zoom factor for over-canopy images
THRESHOLD = 2000      # Minimum area for a blob to be considered valid

# ─── PATHS ───────────────────────────────────────────────────────────
BASE_PATH = Path(f"/Users/at/Orbit_Red_Blob/Data/UKE_Plot_14_DD_{BASE_DATE}")
GT_IMAGE_CSV = BASE_PATH / f"Input/Geo_Tag/GT_UnderCanopy_{GT_DATE}.csv"
GT_VIDEO_CSV = BASE_PATH / f"input/Geo_Tag/GT_OverCanopy_{GT_DATE}.csv"
INPUT_IMAGE_FOLDER = BASE_PATH / "Input/Ground_Truth_Pictures"
VIDEO_INPUT_DIR = BASE_PATH / "Input/Orbit_Videos"
SRT_PATH = VIDEO_INPUT_DIR
VIDEO_NAME = "VIDEO_NAME2"

MODEL_PATH = Path("/Users/at/Orbit_Red_Blob/models/TR2_Ripe_V8.pt")
CROWN_MODEL_PATH = Path("/Users/at/Orbit_Red_Blob/models/orbit_crownv8.pt")

OUTPUT_BASE = BASE_PATH / "output_hsv/Trees"
GT_LOG_CSV = BASE_PATH / "output_hsv/Analysis/GT_log.csv"
LOG_CSV_PATH = BASE_PATH / "output_hsv/Analysis/image_log.csv"
FINAL_OUTPUT_PATH = BASE_PATH / "output_hsv/Analysis/final_analysis.csv"

# # ─── HSV THRESHOLDS 1 ──────────────────────────────────────────────────
# lower_red1 = np.array([0, 100, 20])
# upper_red1 = np.array([10, 255, 150])
# lower_red2 = np.array([160, 100, 25])
# upper_red2 = np.array([179, 255, 188])


# ─── HSV THRESHOLDS 2 ──────────────────────────────────────────────────
lower_red1 = np.array([0, 50, 50])
upper_red1 = np.array([10, 255, 255])
lower_red2 = np.array([160, 50, 50])
upper_red2 = np.array([179, 255, 255])

# ─── SECTION MAP ─────────────────────────────────────────────────────
SECTION_MAP = {
    1: ["30", "31", "y+", "1"],
    2: ["2", "3", "4", "5"],
    3: ["6", "7", "8", "9"],
    4: ["10", "11", "12", "13"],
    5: ["14", "15", "16", "17"],
    6: ["18", "19", "20", "21"],
    7: ["22", "23", "24", "25"],
    8: ["26", "27", "28", "29"],
}
OPPOSITE_SECTIONS = {1: 5, 2: 6, 3: 7, 4: 8, 5: 1, 6: 2, 7: 3, 8: 4}
image_to_section = {img: sec for sec, imgs in SECTION_MAP.items() for img in imgs}

# ─────────────────────────────────────────────────────────────────────
# SECTION 1: Under-canopy image detection
# ─────────────────────────────────────────────────────────────────────
def process_gt_images():
    model = YOLO(str(MODEL_PATH))
    all_imgs = {f.stem.upper(): f for f in INPUT_IMAGE_FOLDER.glob("*.jpg")}
    all_imgs.update({f.stem.upper(): f for f in INPUT_IMAGE_FOLDER.glob("*.JPG")})
    df = pd.read_csv(GT_IMAGE_CSV)
    gt_log_entries, count_by_tree = [], {}

    for _, row in df.iterrows():
        tree_num, image_name_raw = row.get("Tree"), row.get("IMG")
        if pd.isna(tree_num) or pd.isna(image_name_raw): continue

        point_name = f"point{int(tree_num)}"
        image_name = image_name_raw.strip().upper()
        count_by_tree.setdefault(point_name, 0)
        count_by_tree[point_name] += 1
        suffix = count_by_tree[point_name]
        if image_name not in all_imgs:
            print(f"❌ Image not found: {image_name}")
            continue

        input_path = all_imgs[image_name]
        output_dir = OUTPUT_BASE / point_name / "predictions"
        output_dir.mkdir(parents=True, exist_ok=True)
        out_name = f"{image_name}_{suffix}.jpg"
        output_path = output_dir / out_name

        try:
            results = model(str(input_path), verbose=False)[0]
            img = cv2.imread(str(input_path))
            ripe, unripe = 0, 0

            for box in results.boxes:
                cls = int(box.cls[0])
                label = model.names[cls].lower()
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                color = (0, 0, 255) if label == "ripe" else (255, 0, 0)
                if label == "ripe": ripe += 1
                elif label == "unripe": unripe += 1
                cv2.rectangle(img, (x1, y1), (x2, y2), color, 40)

            cv2.imwrite(str(output_path), img)
            gt_log_entries.append({"point": point_name, "image_name": out_name, "Ripe": ripe, "Unripe": unripe})

        except Exception as e:
            print(f"⚠️ Error processing {image_name}: {e}")
            gt_log_entries.append({"point": point_name, "image_name": out_name, "Ripe": 0, "Unripe": 0})

    GT_LOG_CSV.parent.mkdir(parents=True, exist_ok=True)  # Create the directory if needed
    pd.DataFrame(gt_log_entries).to_csv(GT_LOG_CSV, index=False)
    print(f"✅ GT log saved to: {GT_LOG_CSV}")

# ─────────────────────────────────────────────────────────────────────
# SECTION 2: Over-canopy video frame extraction
# ─────────────────────────────────────────────────────────────────────
def detect_crown_center(image, model, conf_thresh=0.5, previous_center=None):
    with contextlib.redirect_stdout(open(os.devnull, 'w')):
        results = model(image, verbose=False)
    boxes = results[0].boxes

    if boxes is None or len(boxes.xyxy) == 0:
        return None

    h, w = image.shape[:2]
    x_min_valid = 0.25 * w
    x_max_valid = 0.75 * w
    y_min_valid = 0.25 * h
    y_max_valid = 0.75 * h

    best_box = None
    max_conf = -1

    for box in boxes:
        if box.conf < conf_thresh:
            continue

        x1, y1, x2, y2 = box.xyxy[0].tolist()
        cx = (x1 + x2) / 2
        cy = (y1 + y2) / 2

        if not (x_min_valid <= cx <= x_max_valid and y_min_valid <= cy <= y_max_valid):
            continue

        if box.conf > max_conf:
            max_conf = box.conf
            best_box = (cx, cy)

    if best_box:
        cx, cy = best_box
        return (cx / w, cy / h)
    else:
        return None
    
# ─── ZOOM FUNCTION ────────────────────────────────────────────────
def apply_zoom(image, center, zoom_factor):
    h, w = image.shape[:2]
    cx = int(center[0] * w)
    cy = int(center[1] * h)

    new_w = int(w / zoom_factor)
    new_h = int(h / zoom_factor)

    x1 = max(cx - new_w // 2, 0)
    y1 = max(cy - new_h // 2, 0)
    x2 = min(cx + new_w // 2, w)
    y2 = min(cy + new_h // 2, h)

    cropped = image[y1:y2, x1:x2]
    zoomed = cv2.resize(cropped, (w, h), interpolation=cv2.INTER_LINEAR)
    return zoomed

def process_overcanopy_videos():
    print("🔍 Processing over-canopy videos...")
    with open(GT_VIDEO_CSV, 'r') as f:
        lines = f.readlines()

    if len(lines) < 2:
        raise ValueError("CSV file has insufficient rows.")

    # second_line = lines[1]
    # if not re.search(r"Point\\s*\\d+", second_line, re.IGNORECASE):
    #     print("⚠️ Skipping processing — 'Point X' pattern not found in second line.")
    #     return

    df = pd.read_csv(GT_VIDEO_CSV, skiprows=[1])
    model = YOLO(str(CROWN_MODEL_PATH))
    log_data = []
    summary_failures = []

    for _, row in tqdm(df.iterrows(), total=len(df)):
        try:
            point = str(row["point"]).strip()
            video_name_raw = row.get(VIDEO_NAME, "")
            if pd.isna(video_name_raw) or str(video_name_raw).strip() == "":
                summary_failures.append((f"{point}", "Empty VIDEO_NAME, skipping"))
                continue
            video_name = str(video_name_raw).strip().upper()
        except Exception as e:
            summary_failures.append((f"{point}", f"Malformed row: {e}"))
            continue

        video_path = VIDEO_INPUT_DIR / f"{video_name.lower()}.mp4"
        if not video_path.exists():
            summary_failures.append((video_name, "Video not found"))
            continue

        output_folder = OUTPUT_BASE / point.lower().replace(" ", "") / "Orbits"
        images_folder = output_folder / "images"
        images_folder.mkdir(parents=True, exist_ok=True)

        cap = cv2.VideoCapture(str(video_path))
        if not cap.isOpened():
            summary_failures.append((video_name, "Could not open video"))
            continue

        fps = cap.get(cv2.CAP_PROP_FPS)
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        duration = total_frames / fps

        if duration <= TARGET_TIME:
            summary_failures.append((video_name, f"Video too short ({duration:.2f}s)"))
            cap.release()
            continue

        # ─── IMAGE EXTRACTION ─────────────────────────────────────────
        zoom_center = None
        total_images = 4 * (images_per_quadrant + 1)
        timestamps = [TARGET_TIME + i * ((duration - TARGET_TIME) / total_images) for i in range(total_images)]

        saved_any_frame = False

        for i, ts in enumerate(timestamps):
            cap.set(cv2.CAP_PROP_POS_MSEC, ts * 1000)
            success, frame = cap.read()

            # EDIT: save the images into the file
            # if success:
            #     raw_frame_name = f"{video_name}_raw_{i}.jpg" if i > 0 else f"{video_name}_raw_y+.jpg"
            #     raw_out_path = images_folder / raw_frame_name
            #     cv2.imwrite(str(raw_out_path), frame, [int(cv2.IMWRITE_JPEG_QUALITY), 95])

            if not success:
                summary_failures.append((video_name, f"Failed to read frame at t={ts:.2f}s"))
                continue

            detected_center = detect_crown_center(frame, model, previous_center=zoom_center)
            if detected_center is not None:
                zoom_center = detected_center
            elif zoom_center is None:
                zoom_center = (0.5, 0.5)

            zoomed = apply_zoom(frame, zoom_center, ZOOM_FACTOR)

            # 🔲 Draw black X to indicate zoom center
            zh, zw = zoomed.shape[:2]
            x = int(zoom_center[0] * zw)
            y = int(zoom_center[1] * zh)
            cv2.line(zoomed, (x - 30, y - 30), (x + 30, y + 30), (0, 0, 0), 3)
            cv2.line(zoomed, (x - 30, y + 30), (x + 30, y - 30), (0, 0, 0), 3)

            if i == 0:
                fname = f"{video_name}_zoomed_y+.jpg"
            else:
                fname = f"{video_name}_zoomed_{i}.jpg"

            out_path = images_folder / fname
            cv2.imwrite(str(out_path), zoomed, [int(cv2.IMWRITE_JPEG_QUALITY), 100])
            saved_any_frame = True

            log_data.append({
                "point": point,
                "video_name": video_name,
                "image_name": fname,
                "timestamp_sec": round(ts, 2)
            })

        cap.release()

        if not saved_any_frame:
            summary_failures.append((video_name, "No frames successfully extracted"))
            print(f"⚠️ No frames saved for {video_name}. Check video duration and timestamps.")

    # ─── SAVE LOG CSV (Update-Only) ───────────────────────────────────
    log_df = pd.DataFrame(log_data)
    LOG_CSV_PATH.parent.mkdir(parents=True, exist_ok=True)

    if LOG_CSV_PATH.exists():
        existing_log_df = pd.read_csv(LOG_CSV_PATH)
        updated_log_df = existing_log_df[~existing_log_df['video_name'].isin(log_df['video_name'].unique())]
        combined_log_df = pd.concat([updated_log_df, log_df], ignore_index=True)
    else:
        combined_log_df = log_df

    combined_log_df.to_csv(LOG_CSV_PATH, index=False)
    print(f"\n📝 Image log updated and saved to: {LOG_CSV_PATH}")

    if summary_failures:
        print("\n📊 Summary of Videos Not Processed:")
        for vid, reason in summary_failures:
            print(f"  - {vid}: {reason}")
    else:
        print("\n✅ All videos processed successfully.")


# ─────────────────────────────────────────────────────────────────────
# SECTION 3: Heading recalibration
# ─────────────────────────────────────────────────────────────────────
def calculate_heading(lat1, lon1, lat2, lon2):
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    dlon = lon2 - lon1
    x = math.sin(dlon) * math.cos(lat2)
    y = math.cos(lat1)*math.sin(lat2) - math.sin(lat1)*math.cos(lat2)*math.cos(dlon)
    bearing = math.atan2(x, y)
    return (math.degrees(bearing) + 360) % 360

def extract_gps_from_srt(srt_file):
    coords, current_time = [], None
    with open(srt_file, 'r') as f:
        for line in f:
            time_match = re.match(r'(\d{2}):(\d{2}):(\d{2})', line)
            if time_match:
                h, m, s = map(int, time_match.groups())
                current_time = h * 3600 + m * 60 + s

            if 'latitude:' in line and 'longitude:' in line and current_time:
                lat_match = re.search(r'\[latitude:\s*([-\d.]+)\]', line)
                lon_match = re.search(r'\[longitude:\s*([-\d.]+)\]', line)
                if lat_match and lon_match and current_time is not None:
                    lat = float(lat_match.group(1))
                    lon = float(lon_match.group(1))
                    coords.append((lat, lon, current_time))
    return coords

def recalibrate_heading():
    gt_df, log_df = pd.read_csv(GT_VIDEO_CSV), pd.read_csv(LOG_CSV_PATH)
    gt_df[VIDEO_NAME] = gt_df[VIDEO_NAME].astype(str).str.strip().str.replace(".MP4", "", case=False)
    log_df["video_name"] = log_df["video_name"].astype(str).str.strip().str.replace(".MP4", "", case=False)

    total_bins = (1 + images_per_quadrant) * 4
    bin_width = 360 / total_bins

    for idx, row in log_df.iterrows():
        if "y+" not in row["image_name"]: continue
        gt_match = gt_df[(gt_df[VIDEO_NAME].str.upper() == row["video_name"].upper()) & (gt_df["point"].astype(str).str.lower() == row["point"].lower())]
        if gt_match.empty: continue

        tree_lat = float(gt_match.iloc[0]["Latitude"])
        tree_lon = float(gt_match.iloc[0]["Longitude"])
        srt_file = SRT_PATH / f"{row['video_name'].upper()}.SRT"
        if not srt_file.exists(): continue

        coords = extract_gps_from_srt(srt_file)
        closest = min(coords, key=lambda x: abs(x[2] - TARGET_TIME), default=None)
        if not closest: continue

        heading = calculate_heading(tree_lat, tree_lon, closest[0], closest[1])
        adj = (heading - REFERENCE_ANGLE + 360) % 360
        index = round(adj / bin_width) % total_bins
        log_df.loc[idx, "heading"] = round(heading, 2)
        log_df.loc[idx, "adjustment_angle"] = round(adj, 2)
        log_df.loc[idx, "adjustment_index"] = index

    # Save CSV
    log_df.to_csv(LOG_CSV_PATH, index=False)
    print(f"✅ Updated log with headings → {LOG_CSV_PATH}")

    # Summary
    #print(f"\n✅ Completed processing {len(tasks_images)} / {len(tasks_images)} images.")
    print(f"📝 CSV log saved to: {LOG_CSV_PATH}")


# ─────────────────────────────────────────────────────────────────────
# SECTION 4: Blob extraction
# ─────────────────────────────────────────────────────────────────────
def process_red_blobs():
    df = pd.read_csv(LOG_CSV_PATH)
    df["max_blob_grey"] = df.get("max_blob_grey", "")
    df["max_blob_red"] = df.get("max_blob_red", "")
    tasks = []
    #processed = 0
    empty_folders =[]

    for folder in OUTPUT_BASE.glob("**/Orbits/images"):
        task_count = 0
        for img_path in sorted(folder.glob("*.jpg")):  # 🔁 Now sorted alphabetically
            if "_grey" in img_path.name or "_blob" in img_path.name:
                continue
            matches = df.index[df["image_name"] == img_path.name].tolist()
            if matches:
                tasks.append((img_path, matches[0]))
                task_count +=1
        if task_count == 0:
            empty_folders.append(str(folder.parent))  # One level up from /images


    for img_path, idx in tqdm(tasks, desc="🧠 Red-blob mask"):
        img = cv2.imread(str(img_path))
        if img is None: continue

        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        mask = cv2.bitwise_or(cv2.inRange(hsv, lower_red1, upper_red1), cv2.inRange(hsv, lower_red2, upper_red2))
        inv_mask = cv2.bitwise_not(mask)
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        max_blob_grey = max([cv2.contourArea(c) for c in contours], default=0)

        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        gray_colored = cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR)
        red = cv2.bitwise_and(img, img, mask=mask)
        grey = cv2.bitwise_and(gray_colored, gray_colored, mask=inv_mask)
        result = cv2.add(red, grey)
        grey_path = img_path.with_name(img_path.stem + "_grey.jpg")
        cv2.imwrite(str(grey_path), result)

        # Generate blob output
        grey_img = cv2.imread(str(grey_path))
        mask = cv2.bitwise_or(cv2.inRange(cv2.cvtColor(grey_img, cv2.COLOR_BGR2HSV), lower_red1, upper_red1),
                              cv2.inRange(cv2.cvtColor(grey_img, cv2.COLOR_BGR2HSV), lower_red2, upper_red2))
        inv = cv2.bitwise_not(mask)
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        max_blob_red = max([cv2.contourArea(c) for c in contours], default=0)

        white = np.full_like(grey_img, 255)
        blob = cv2.add(cv2.bitwise_and(grey_img, grey_img, mask=mask),
                       cv2.bitwise_and(white, white, mask=inv))
        blob_path = img_path.with_name(img_path.stem + "_blob.jpg")
        cv2.imwrite(str(blob_path), blob)

        df.at[idx, "max_blob_grey"] = int(max_blob_grey)
        df.at[idx, "max_blob_red"] = int(max_blob_red)

    df.to_csv(LOG_CSV_PATH, index=False)
    print(f"✅ Red-blob metrics updated → {LOG_CSV_PATH}")

    if empty_folders:
        print("\n📭 The following folders had no valid images for processing:")
    for folder in empty_folders:
        print(f"  - {folder}")

# ─────────────────────────────────────────────────────────────────────
# SECTION 5: Final analysis and scoring
# ─────────────────────────────────────────────────────────────────────
def perform_final_analysis():
    gt_log = pd.read_csv(GT_LOG_CSV)
    image_log = pd.read_csv(LOG_CSV_PATH)

    opposite_map = {"8": "4", "3": "7", "1": "5", "2": "6", "4": "8", "7": "3", "5": "1", "6": "2"}

    def extract_img_number(name):
        return name.split("_")[-1].split(".")[0]

    def count_ground_truth(group):
        group['img_num'] = group['image_name'].apply(extract_img_number).astype(str)
        group['Ripe'] = group['Ripe'].astype(int)

        if (group['Ripe'] == 2).any():
            return 2

        ripe_imgs = group[group['Ripe'] == 1]['img_num'].tolist()
        all_imgs = group['img_num'].tolist()

        ripe_flags = [img in ripe_imgs for img in all_imgs]
        ripe_flags_looped = ripe_flags + ripe_flags[:3]

        max_consecutive = 0
        current = 0
        for val in ripe_flags_looped:
            if val:
                current += 1
                max_consecutive = max(max_consecutive, current)
            else:
                current = 0

        if max_consecutive >= 5:
            return 2
        elif max_consecutive >= 1:
            for img in ripe_imgs:
                if img in opposite_map and opposite_map[img] in ripe_imgs:
                    return 2
            return 1

        return 0

    final_rows = []
    for point, group in gt_log.groupby("point"):
        gt_count = count_ground_truth(group)
        final_rows.append({"pointX": point, "ground_truth": gt_count, "orbits": "n/a"})
    final_df = pd.DataFrame(final_rows)

    def extract_image_number(name):
        return Path(name).stem.split("_")[-1]

    image_log["pointX"] = image_log["point"]
    image_log["image_num"] = image_log["image_name"].apply(extract_image_number)

    orbit_counts = {}
    for point, group in image_log.groupby("pointX"):
        high_spikes = group[group["max_blob_grey"] > THRESHOLD].copy()
        high_spikes["section"] = high_spikes["image_num"].map(image_to_section)
        sections = high_spikes["section"].dropna().astype(int).tolist()
        unique_sections = set(sections)

        if len(unique_sections) >= 5:
            orbit_value = 2
        elif len(unique_sections) >= 1:
            found_opposites = any(OPPOSITE_SECTIONS.get(sec) in unique_sections for sec in unique_sections)
            orbit_value = 2 if found_opposites else 1
        else:
            orbit_value = 0

        orbit_counts[point] = orbit_value

    # ✅ Vectorized assignment (clean and safe)
    final_df["orbits"] = final_df["pointX"].map(orbit_counts).fillna("n/a")

    def binary_check(row):
        try:
            if row["orbits"] == "n/a":
                return "n/a"
            gt = int(row["ground_truth"])
            orb = int(row["orbits"])
            return "good" if gt > 0 and orb > 0 else "bad"
        except:
            return "n/a"

    final_df["binary_check"] = final_df.apply(binary_check, axis=1)

    valid_binary = final_df[final_df["binary_check"] != "n/a"]
    total_rows = len(valid_binary)
    good_matches = (valid_binary["binary_check"] == "good").sum()
    binary_accuracy_percent = 100 * good_matches / total_rows if total_rows > 0 else 0

    def row_accuracy(row):
        try:
            if row["orbits"] == "n/a":
                return "n/a"
            gt = int(row["ground_truth"])
            orb = int(row["orbits"])
            if gt == 0 and orb == 0:
                return 1.0
            elif gt == 0 or orb == 0:
                return 0.0
            return min(gt, orb) / max(gt, orb)
        except:
            return 0.0

    final_df["accuracy_score"] = final_df.apply(row_accuracy, axis=1)

    valid_numeric = final_df[final_df["orbits"] != "n/a"].copy()
    valid_numeric["orbits"] = valid_numeric["orbits"].astype(int)

    total_gt = valid_numeric["ground_truth"].astype(int).sum()
    total_orb = valid_numeric["orbits"].sum()

    # New total accuracy: average of per-row accuracy_score
    valid_scores = final_df[final_df["accuracy_score"] != "n/a"]
    total_accuracy_percent = 100 * valid_scores["accuracy_score"].astype(float).mean() if not valid_scores.empty else 0.0

    print(f"[✅] Binary Match Accuracy: {binary_accuracy_percent:.2f}% ({good_matches}/{total_rows})")
    print(f"[✅] Overall Count Accuracy: {total_accuracy_percent:.2f}%")
    print(f"[✅] Total Ground Truth (Ripe FFB): {total_gt}")
    print(f"[✅] Total Orbit Count (Ripe FFB):  {total_orb}")

    final_df.to_csv(FINAL_OUTPUT_PATH, index=False)
    print(f"\n📊 Summary of Ground Truth and Orbit Analysis:\n{final_df}")

# ─── MAIN ─────────────────────────────────────────────────────────────
if __name__ == "__main__":
    if PROCESS_IMAGES:
        process_gt_images()
    if PROCESS_VIDEOS:
        process_overcanopy_videos()
    if RECALIBRATE_HEADING:
        recalibrate_heading()
    if EXTRACT_REDBLOB:
        process_red_blobs()
    if FINAL_ANALYSIS:
        perform_final_analysis()
    print("\n✅ All enabled processing steps completed.")

